In [ ]:
# @title  Whisper Turbo Fine-Tuning with LoRA (Whisper large-v3-turbo)

# 1. Install dependencies
# !pip install -q transformers librosa accelerate peft bitsandbytes

!pip install -q jiwer evaluate pythainlp

# !pip install --upgrade datasets==3.6.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 86.3 MB/s eta 0:00:00


In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
import pandas as pd
import librosa
import glob
import os
import librosa
from tqdm import tqdm

In [ ]:
import pandas as pd
import librosa
import glob
import os
import librosa
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# train = pd.read_csv(r"/content/drive/MyDrive/SignLang/train_all_old.csv")
# eval = pd.read_csv(r"/content/drive/MyDrive/SignLang/eval_all.csv")

In [ ]:
# def transform_path(x):
#   name = x.split("/")[-1]
#   if "common" in x:
#     return f"/content/drive/MyDrive/SignLang/dataset_all/train/th_train_0/{name}"
#   else:
#     return f"/content/drive/MyDrive/SignLang/dataset_all/train/train/{name}"

In [ ]:
# train["path"] = train["path"].apply(lambda x:transform_path(x))
# eval["path"] = eval["path"].apply(lambda x:transform_path(x))

In [ ]:
# real = 0
# duration = 0
# for i in tqdm(train["path"],total=len(train)):
#     if os.path.exists(i):
#         # wav,fs = librosa.load(i)
#         # duration +=librosa.get_duration(y=wav,sr=fs)
#         real+=1
#     else:
#       print(i)

100%|██████████| 35425/35425 [00:08<00:00, 4041.64it/s]


In [ ]:
# train.to_csv("/content/drive/MyDrive/SignLang/train_all.csv",index=False)
# eval[["path","transcription"]].to_csv("/content/drive/MyDrive/SignLang/eval_all.csv",index=False)

In [ ]:
# !huggingface-cli download "openai/whisper-large-v3" --repo-type model --local-dir "/content/drive/MyDrive/whisper-large-v3"

In [ ]:
# 2. Enable GPU
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# 3. Load dataset (example: Common Voice English; replace with your dataset)
from datasets import load_dataset
# dataset = load_dataset("mozilla-foundation/common_voice_17_0", "th",trust_remote_code=True, split = {
#     "train": "train[:30%]",
#     "validation": "validation[:30%]"
# })
# save_path  = r"F:\Hybridmodel-project\dataset\asr\cv17"
base_path = "/content/drive/MyDrive/SignLang"
trian_path = os.path.join(base_path,"train_all.csv")
eval_path = os.path.join(base_path,"eval_all.csv")
dataf = {"train":trian_path,"eval":eval_path}
dataset = load_dataset("csv", data_files=dataf,split={"train":"train","eval":"eval[:10%]"})
# eval = load_dataset("csv", data_files=r"eval_all.csv")




# dataset.save_to_disk(save_path)
model_path = "/content/drive/MyDrive/whisper-large-v3"
# Preprocessing
import librosa
from transformers import WhisperFeatureExtractor, WhisperTokenizer
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path)




Using device: cuda


Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [ ]:
# train2 = train.rename_columns({"path":"input_features","transcription":"labels"})

train = dataset["train"].rename_columns({"path":"input_features","transcription":"labels"})
eval = dataset["eval"].rename_columns({"path":"input_features","transcription":"labels"})

In [ ]:
# 4. Load model with PEFT-LoRA
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType,PeftModel

peft_config = LoraConfig(r=16, lora_alpha=132, lora_dropout=0.05,target_modules = ["q_proj","v_proj"])
model = get_peft_model(model, peft_config)
# peft_model_id = "/content/drive/MyDrive/whisper-large-v3-output/r16/checkpoint-200" # e.g., "artificialguybr/3DRedmond-V1"

# model = PeftModel.from_pretrained(model, peft_model_id)



In [ ]:
import gc
class Datacollator:
    def __init__(self,feature_extractor,label_pad_token_id,tokenizer):
        self.feature_extractor = feature_extractor
        self.label_pad_token_id = label_pad_token_id
        self.tokenizer = tokenizer
        self.padding= "longest"
    def __call__(self,data):
        # print(len(data))
        input_features = [{"input_features": librosa.load(f["input_features"],sr=16000)[0]} for f in data]
        label_features = [{"input_ids": tokenizer(f["labels"],return_tensors="pt").input_ids[0]} for f in data]

        # audio_array,fs = librosa.load(path,sr=16000)

        input_features = [{"input_features": feature_extractor(f["input_features"], sampling_rate = 16000, return_tensors="pt").input_features[0]} for f in input_features]

        # print([i.size() for i in input_features["input_features"]])

        batch_inputs = self.feature_extractor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt"
        )
        batch_labels = self.tokenizer.pad(
            label_features,
            padding=self.padding,
            return_tensors="pt"
        )



        labels = batch_labels["input_ids"].masked_fill(
            batch_labels["input_ids"] == self.tokenizer.pad_token_id, -100
        )

        batch = {
            "input_features": batch_inputs["input_features"],
            "labels": labels
        }
        del input_features,label_features,batch_inputs,batch_labels,labels
        torch.cuda.empty_cache()
        gc.collect()

        return  batch


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wayupuk-som (wayupuk-som-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# 5. Training setup
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq,EarlyStoppingCallback
import os
os.environ["WANDB_DISABLED"] = "False"
os.environ["WANDB_PROJECT"] = "whisper-finetuning"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint-large-r16"  # log all model checkpoints
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/whisper-large-v3-output/r16_full_data",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=3e-5,
    max_steps=4000,
    # num_train_epochs=5,
    optim="adamw_torch",
    logging_steps=10,
    save_steps=200,
    fp16=True,
    push_to_hub=False,
    eval_strategy="steps",
    eval_steps=200,
    label_names=["labels"],
    report_to="wandb"
)

data_collator = Datacollator(feature_extractor, label_pad_token_id=tokenizer.pad_token_id,tokenizer=tokenizer)
early_stop = EarlyStoppingCallback(
        early_stopping_patience=20,  # Wait 5 evaluation steps/epochs for improvement
        early_stopping_threshold=0.01 # Optional: minimum change to be considered an improvement
    )
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    tokenizer=feature_extractor,
    data_collator=data_collator,
    # compute_metrics=compute_metrics
)

# 9. Train
trainer.train()

# # 10. Evaluate
# metrics = trainer.evaluate()
# print("Evaluation Results:", metrics)

# 11. Save final model
model.save_pretrained("/content/drive/MyDrive/whisper-large-v3-output/r16_full_data/full")
# processor.save_pretrained("whisper_turbo_finetuned")

### แชณ์เฉยๆกันจอดับ

/tmp/ipython-input-1791888437.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Received unrecognized `WANDB_LOG_MODEL` setting value=checkpoint-large-r16; so disabling `WANDB_LOG_MODEL`
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: wayupuk-som (wayupuk-som-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
200,0.251200,0.166899
400,0.188900,0.150068
600,0.214100,0.143989
800,0.169800,0.139474
1000,0.200500,0.135033
1200,0.183700,0.133305
1400,0.159100,0.130208
1600,0.125200,0.127399


In [ ]:
# model.save_pretrained("/content/drive/MyDrive/whisper-large-v3-output/full")

# Test pipeline

In [ ]:
!pip install -q jiwer evaluate pythainlp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 105.6 MB/s eta 0:00:00


In [ ]:
from peft import PeftModel
from transformers import WhisperForConditionalGeneration,WhisperProcessor,pipeline
import pandas as pd
import librosa
import glob
import os
import librosa
from tqdm import tqdm

import torch
from datasets import load_dataset,Dataset
from pythainlp.tokenize import word_tokenize
from google.colab import drive
drive.mount('/content/drive')

# if torch.cuda.is_available():
#   device = "cuda"
# else:
#   device = "cpu"
# model_path = r"/content/drive/MyDrive/whisper-large-v3"
# model = base_model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)
# peft_model_id = "/content/drive/MyDrive/whisper-large-v3-output/r16/full" # e.g., "artificialguybr/3DRedmond-V1"

# model = PeftModel.from_pretrained(base_model, peft_model_id)
# processor = WhisperProcessor.from_pretrained(model_path)

Mounted at /content/drive


In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device_map="auto"
)
generate_kwargs={"task": "transcribe",
                 "language": "thai",
                 "no_repeat_ngram_size": 3, # prevent repeating 3-grams
                  "length_penalty":1}


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu


In [ ]:

dataf = {"train":"/content/drive/MyDrive/SignLang/train_all.csv","eval":"/content/drive/MyDrive/SignLang/eval_all.csv"}
dicta = {"inference":[],
         "label":[]}

dataset = load_dataset("csv", data_files=dataf,split={"train":"train","eval":"eval[80%:]"})
evala = dataset["eval"].rename_columns({"path":"input_features","transcription":"labels"})

batch = 1
init = 0
datas = evala["input_features"][:]
labels = evala["labels"][:]
# for i in tqdm(range(0,len(datas)//batch)):
#   data = datas[i * batch :(i+1) * batch]
#   label = labels[i * batch :(i+1) * batch]


#   # pipe(datas[0], generate_kwargs=generate_kwargs,batch_size = 8)
#   dicta["inference"] += pipe(datas, generate_kwargs=generate_kwargs,batch_size = 8)
#   dicta["label"] += labels



# test = pd.DataFrame(dicta)
# test.to_csv("test.csv",index=False)

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import pipeline
import torch
MODEL_NAME = "biodatlab/distill-whisper-th-large-v3"
lang = "th"
device = 0 if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
    return_timestamps=True,
)
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(
    language=lang,
    task="transcribe"
)
# result = pipe("audio.mp3", return_timestamps=True)
# text, timestamps = result["text"], result["chunks"]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.24G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [ ]:
# datas = eval["input_features"][:]
# labels = eval["labels"][:]
# # for i in tqdm(range(0,len(datas)//batch)):
#   data = datas[i * batch :(i+1) * batch]
#   label = labels[i * batch :(i+1) * batch]


  # pipe(datas[0], generate_kwargs=generate_kwargs,batch_size = 8)
dicta["inference"] += pipe(datas[0],batch_size = 8)
dicta["label"] += labels

In [ ]:
import numpy as np
i = np.random.randint(len(datas))

print(pipe(datas[i],batch_size = 1)["text"])
print(labels[i])

ความแตกต่างที่สำคัญคือ จำนวนของประสบการณ์ กล่าวคือ ปัจจัยการผลิต
ความแตกต่างที่สำคัญคือจำนวนของประสบการณ์กล่าวคือปัจจัยการผลิต


1

In [ ]:
# dicta["inference"]
dicta["infer"] = [i["text"] for i in dicta["inference"]]

In [ ]:
import pandas as pd
test = pd.DataFrame(dicta)
test.to_csv("/content/drive/MyDrive/test_new.csv",index=False)

In [ ]:
import string
def cut_word_th(df):
    label_cuts, pred_cuts = [], []

    df["completion_cut"] = df.label.apply(
        lambda x : str(" ".join(word_tokenize(
            clean_text(x).lower().replace(" ", ""),
            engine="newmm")
            )
        )
    )

    df["prediction_cut"] = df.infer.apply(
        lambda x : str(" ".join(word_tokenize(
            clean_text(x).lower().replace(" ", ""),
            engine="newmm")
            )
        )
    )
    return df

In [ ]:
def compute_wer(df):
    df = cut_word_th(df)
    s = df.completion_cut.to_list()
    t = df.prediction_cut.to_list()
    wer = wer_metric.compute(references=s, predictions=t)
    cer = cer_metric.compute(references=s, predictions=t)
    return wer,cer

# Clean text funciton
def clean_text(txt: str) -> str:
    # Clean text credit by thonburian whisper

    # Remove zero-width and non-breaking space.
    txt = txt.replace("\u200b", " ")
    txt = txt.replace("\xa0", " ")

    # Normalize characters and correct common typing errors
    txt = txt.replace("เเ", "แ")
    txt = txt.replace("ํา", "ำ")
    txt = txt.replace("่ำ", "่ำ")
    txt = txt.replace("ำ้", "้ำ")
    txt = txt.replace("ํ่า", "่ำ")

    # Replace special underscore and dash
    txt = txt.replace("▁", "_")
    txt = txt.replace("—", "-")
    txt = txt.replace("–", "-")
    txt = txt.replace("−", "-")

    # Replace special characters
    txt = txt.replace("’", "'")
    txt = txt.replace("‘", "'")
    txt = txt.replace("”", '"')
    txt = txt.replace("“", '"')

    # Remove punctuation
    chars_to_remove = list(string.punctuation)
    for char in chars_to_remove:
        txt = txt.replace(char, "")

    txt = txt.strip()
    return txt

import evaluate
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/test_new.csv")
df

,inference,label,infer
0,{'text': 'สตาร์ทไมโครอิเล็กทรอนิกส์ ประเทศไทย'},สตาร์ส ไมโครอิเล็กทรอนิกส์ ประเทศไทย,สตาร์ทไมโครอิเล็กทรอนิกส์ ประเทศไทย
1,{'text': 'โทรหาคุณพ่อ'},โทรหาคุณพ่อ,โทรหาคุณพ่อ
2,{'text': 'เป็นยังไงบ้างเพื่อนพร้อมที่จะไปหรือย...,เป็นยังไงบ้างเพื่อน พร้อมที่จะไปหรือยัง ?,เป็นยังไงบ้างเพื่อนพร้อมที่จะไปหรือยัง
3,{'text': 'ม้าตีนปลาย'},ม้าตีนปลาย,ม้าตีนปลาย
4,{'text': 'จนตรอบ'},จนตรอก,จนตรอบ
...,...,...,...
731,{'text': 'เป็นรัฐมนตรีช่วยว่าการกระทรวงการคลัง'},เป็นรัฐมนตรีช่วยว่าการกระทรวงการคลัง,เป็นรัฐมนตรีช่วยว่าการกระทรวงการคลัง
732,{'text': 'อย่าโยนทาโรคลงอ่าอาบน้ำ'},อย่าโยนทารกลงอ่างอาบน้ำ,อย่าโยนทาโรคลงอ่าอาบน้ำ
733,{'text': 'ตอนนั้นคุณคิดว่าเขาอาจจะยากสำหรับฉัน...,ดังนั้นคุณคิดว่าเขาอาจจะยากสำหรับฉันใช่มั้ย,ตอนนั้นคุณคิดว่าเขาอาจจะยากสำหรับฉันใช่ไหม
734,{'text': 'ถ้าคุณอยากจองตัวก็แค่หย่อนเช็คใส่ตู้...,ถ้าคุณอยากจองตั๋วก็แค่หย่อนเช็กใส่ตู้ไปรษณีย์สิ,ถ้าคุณอยากจองตัวก็แค่หย่อนเช็คใส่ตู้ใบรายสนีย์สิ


In [ ]:
new_df = compute_wer(df)
print(new_df)

(0.27890574909168625, 0.08530071986995898)


In [ ]:
new_df = compute_wer(df)
print(new_df)

(0.27890574909168625, 0.08530071986995898)


In [ ]:
import numpy as np
abc = np.random.randint(len(test))
test[["completion_cut","prediction_cut"]].iloc[abc]

,491
completion_cut,โอน ไฟ จาก เครื่อง
prediction_cut,โอน ไฟล์ จาก เครื่อง


,inference,label,infer
0,{'text': 'สตาร์ทไมโครอิเล็กทรอนิกส์ ประเทศไทย'},สตาร์ส ไมโครอิเล็กทรอนิกส์ ประเทศไทย,สตาร์ทไมโครอิเล็กทรอนิกส์ ประเทศไทย
1,{'text': 'โทรหาคุณพ่อ'},โทรหาคุณพ่อ,โทรหาคุณพ่อ
2,{'text': 'เป็นยังไงบ้างเพื่อนพร้อมที่จะไปหรือย...,เป็นยังไงบ้างเพื่อน พร้อมที่จะไปหรือยัง ?,เป็นยังไงบ้างเพื่อนพร้อมที่จะไปหรือยัง
3,{'text': 'ม้าตีนปลาย'},ม้าตีนปลาย,ม้าตีนปลาย
4,{'text': 'จนตรอบ'},จนตรอก,จนตรอบ
...,...,...,...
731,{'text': 'เป็นรัฐมนตรีช่วยว่าการกระทรวงการคลัง'},เป็นรัฐมนตรีช่วยว่าการกระทรวงการคลัง,เป็นรัฐมนตรีช่วยว่าการกระทรวงการคลัง
732,{'text': 'อย่าโยนทาโรคลงอ่าอาบน้ำ'},อย่าโยนทารกลงอ่างอาบน้ำ,อย่าโยนทาโรคลงอ่าอาบน้ำ
733,{'text': 'ตอนนั้นคุณคิดว่าเขาอาจจะยากสำหรับฉัน...,ดังนั้นคุณคิดว่าเขาอาจจะยากสำหรับฉันใช่มั้ย,ตอนนั้นคุณคิดว่าเขาอาจจะยากสำหรับฉันใช่ไหม
734,{'text': 'ถ้าคุณอยากจองตัวก็แค่หย่อนเช็คใส่ตู้...,ถ้าคุณอยากจองตั๋วก็แค่หย่อนเช็กใส่ตู้ไปรษณีย์สิ,ถ้าคุณอยากจองตัวก็แค่หย่อนเช็คใส่ตู้ใบรายสนีย์สิ


# code run

In [ ]:
from peft import PeftModel
from transformers import WhisperForConditionalGeneration,WhisperProcessor,pipeline

import torch
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
model_path = r"/content/drive/MyDrive/whisper-large-v3"
model = base_model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)
peft_model_id = "/content/drive/MyDrive/whisper-large-v3-output/full" # e.g., "artificialguybr/3DRedmond-V1"

model = PeftModel.from_pretrained(base_model, peft_model_id)
processor = WhisperProcessor.from_pretrained(model_path)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device=device,
)
generate_kwargs={"task": "transcribe",
                 "language": "thai",
                 "no_repeat_ngram_size": 3, # prevent repeating 3-grams
                  "length_penalty":1}


In [ ]:
result = pipe(data, generate_kwargs=generate_kwargs)

In [ ]:
import torch
from torchmetrics.functional.audio import dnsmos

# Load audio: reference / processed or only processed (DNSMOS is non-reference)
# Here we only need processed audio for DNSMOS
import librosa
from google.colab import drive
drive.mount('/content/drive')

# audio_path = "/root/se_test/storm/result/files/1188-133604-0012.wav"
# audio, sr = load(audio_path)
# metric = dnsmos.deep_noise_suppression_mean_opinion_score(audio,fs=sr,personalized = False)
# audio_tensor = torch.tensor(audio).unsqueeze(0)  # batch of 1
# score = metric(audio_tensor)


# print("DNSMOS score:", score.item())


ValueError: mount failed